In [ ]:
# Importing necessary libraries:
import torch
import torchvision
import torchvision.transforms as T
from torch import nn
from torchvision.datasets import STL10
from torch.utils.data.dataloader import DataLoader

In [ ]:
# Device agnostic code:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"The current device torch is on: {device}")

In [ ]:
transform = T.Compose([
    T.Resize(size=(224, 224)),
    T.ToTensor()
])

print(f"Transformation pipeline:\n{transform}")

In [ ]:
# Training and testing dataset
train_data = STL10(root = "dataset", split = "train", transform = transform, download = True)
test_data = STL10(root = "dataset", split = "test", transform = transform, download = True)

In [ ]:
print(f"Length of training_data: {len(train_data)}")
print(f"Length of testing data: {len(test_data)}")

In [ ]:
classes = train_data.classes
print(classes)

In [ ]:
# Visualization:
import matplotlib.pyplot as plt

image = train_data[0][0].permute(1,2,0) # [channels, height, width] -> [height, width, channels]
label = train_data[0][1]

# Plot image
plt.figure()
plt.imshow(image)
plt.title(f"{classes[label]},{image.shape}")
plt.axis(False)

In [ ]:
# Coverting train and test datasets into DataLoaders
BATCH_SIZE = 64
train_dataloader = DataLoader(dataset=train_data, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(dataset=test_data, batch_size = BATCH_SIZE, shuffle = False)

print(f"Length of train_dataloader:{len(list(next(iter(train_dataloader)))[0])}")
print(f"Length of test_dataloader:{len(list(next(iter(test_dataloader)))[0])}")

In [ ]:
# Define your ResNet-50 model
class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        # Define the architecture of ResNet-50
        # You can use torchvision.models to load a pre-defined ResNet-50
        # or implement it from scratch.
        self.resnet50 = torchvision.models.resnet50(pretrained=False)
        # Modify the final fully connected layer to match your number of classes
        self.resnet50.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        return self.resnet50(x)

In [ ]:
model = ResNet50(num_classes = len(classes))
model

In [ ]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.01)

In [15]:
epochs = 5
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if (batch_idx + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Batch [{batch_idx + 1}/{len(train_dataloader)}], Loss: {total_loss / (batch_idx + 1):.4f}')

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(train_dataloader):.4f}')

Epoch [1/5], Batch [10/79], Loss: 2.8619
Epoch [1/5], Batch [20/79], Loss: 2.6286
Epoch [1/5], Batch [30/79], Loss: 2.5451
Epoch [1/5], Batch [40/79], Loss: 2.4741
Epoch [1/5], Batch [50/79], Loss: 2.4018
Epoch [1/5], Batch [60/79], Loss: 2.3345
Epoch [1/5], Batch [70/79], Loss: 2.2860
Epoch [1/5], Loss: 2.2487
Epoch [2/5], Batch [10/79], Loss: 1.9485
Epoch [2/5], Batch [20/79], Loss: 1.8902
Epoch [2/5], Batch [30/79], Loss: 1.8655


KeyboardInterrupt: ignored

In [18]:
X, y = next(iter(test_dataloader))
X
# Inferencing
model.eval()
with torch.inference_mode():
  prediction = model(X)

In [20]:
print(prediction.argmax(dim=1))

tensor([5, 4, 0, 0, 5, 5, 5, 4, 5, 4, 0, 8, 4, 0, 4, 5, 5, 2, 0, 1, 1, 5, 4, 5,
        9, 9, 1, 1, 2, 4, 9, 1, 1, 5, 2, 9, 5, 5, 9, 1, 5, 0, 8, 1, 0, 2, 1, 1,
        5, 9, 5, 1, 4, 0, 9, 0, 5, 0, 2, 8, 0, 5, 8, 4])


In [28]:
model_name = "resnet50.pth"

torch.save(model.state_dict(), model_name)